# Use banks to cache prompts with Anthropic API

<a target="_blank" href="https://colab.research.google.com/github/masci/banks/blob/main/cookbook/Prompt_Caching_with_Anthropic.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Prompt caching allows you to store and reuse context within your prompt saving time and money. When using the prompt cache feature from Anthropic, the chat messages have to be expressed in blocks rather than simple text, so that you can designate one of the for being cached.

Let's see how Banks makes this super easy.

In [ ]:
!pip install banks

To simulate a huge prompt, we'll provide Claude with a full book in the context, "Pride and prejudice".

In [ ]:
!curl -O https://www.gutenberg.org/cache/epub/1342/pg1342.txt

Read the whole book and assign to the `book` variable.

In [ ]:
with open("pg1342.txt") as f:
    book = f.read()

With Banks we can define which part of the prompt specifically will be cached. 
Directly from the prompt template text, we can use the `cache_control` built-in filter to tell Anthropic that
we want to cache the prompt up to and including the `{{ book }}` template block.

In [ ]:
import time

import litellm
from litellm import completion

from banks import Prompt


tpl = """
{% chat role="user" %}
Analyze this book:

{# Only this part of the message content (including the book content) will be cached #}
{{ book | cache_control("ephemeral") }}

{# This part won't be cached instead #}

What is the title of this book? Only output the title.
{% endchat %}
"""

p = Prompt(tpl)
# render the prompt in form of a list of Banks' ChatMessage
chat_messages = p.chat_messages({"book": book})
# dump the ChatMessage objects into dictionaries to pass to LiteLLM
messages_dict = [m.model_dump(exclude_none=True) for m in chat_messages]

Let's call the Anthropic API for the first time. We don't expect any difference from a normal call without caching.

In [ ]:
# First call has no cache
start_time = time.time()
response = completion(model="anthropic/claude-3-5-sonnet-20240620", messages=messages_dict)

print(f"Non-cached API call time: {time.time() - start_time:.2f} seconds")
print(response.usage)
print(response)

Now the book content is in the cache, and the difference in time and cost repeating the previous call is obvious.

In [ ]:
# Second call, the book is cached
start_time = time.time()
response = completion(model="anthropic/claude-3-5-sonnet-20240620", messages=messages_dict)

print(f"Cached API call time: {time.time() - start_time:.2f} seconds")
print(response.usage)
print(response)